In [ ]:
# cross attention on channel 's softmax on dim=0
import torch.nn.functional as F
from torch import nn
import torch
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import torch.optim as optim
import numpy as np
import sys
print(torch.__version__)
print(torch.version.cuda)
import math
from sklearn import preprocessing
from torch.autograd import Variable
import pytorch_warmup as warmup
import torch.nn.functional as F
import random
from sklearn.model_selection import KFold, train_test_split
weight_decay=0
learning_rate=1e-3
depth=3
side_depth=3
print("weight_decay",weight_decay)
print("learning_rate",learning_rate)
print("depth",depth)
print("side_depth",side_depth)
'''
WD=np.int(sys.argv[1])
if WD == 2:
    weight_decay=1e-2

if WD == 3:
    weight_decay=5e-2

if WD == 4:
    weight_decay=1e-1
'''
# seed setting
seed = random.randint(0, 999999)
seed = 778539

print("random seed:", seed)

torch.manual_seed(seed)

# Define the number of folds for cross-validation
num_folds = 9

# Shuffle and split the dataset into k folds
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)


# used as position embedding for PPS data
class PositionalEncoding(nn.Module):

    def __init__(self, d_hid, n_position=200):
        super(PositionalEncoding, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_position, d_hid))

    def _get_sinusoid_encoding_table(self, n_position, d_hid):
        ''' Sinusoid position encoding table '''

        def get_position_angle_vec(position):
            return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

        sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

        return torch.FloatTensor(sinusoid_table).unsqueeze(0)

    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()


# used as position embedding for EEG data
class PositionalEncoding2d(nn.Module):

    def __init__(self, d_hid, d_wid, n_position=200):
        super(PositionalEncoding2d, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_position, d_hid, d_wid))

    def _get_sinusoid_encoding_table(self, n_position, d_hid, d_wid):

        def get_position_angle_vec(position):

            return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

        sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])

        sinusoid_table_x = np.zeros_like(sinusoid_table)
        sinusoid_table_y = np.zeros_like(sinusoid_table)
        # horizontal position coding list
        sinusoid_table_x[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # for position as 2i, sin() is used
        sinusoid_table_x[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # for position as 2i+1, cos() is used
        # vertical position coding list
        sinusoid_table_y[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # for position as 2j, sin() is used
        sinusoid_table_y[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # for position as 2j+1, cos() is used

        sinusoid_table = np.zeros((n_position, d_hid, d_wid))

        for i in range(d_hid):
            for j in range(d_wid):
                for k in range(n_position):
                    # multiply two lists to get position encoding table for EEG data
                    sinusoid_table[k][i][j] = sinusoid_table_x[k, i] * sinusoid_table_y[k, j]

        return torch.FloatTensor(sinusoid_table).unsqueeze(0)

    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()


# this module could implement LayerNorm for single-modal data and  multimodal data (seperately).
# the Norm function is invoked before the latter "fn" function.
class PreNorm(nn.Module):
    def __init__(self, dim, side_embedding_dim, fn):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(side_embedding_dim)
        self.fn = fn

    def forward(self, inp, **kwargs):

        if len(inp) == 2:
            x = inp[0]

            y = inp[1]

            return self.fn((self.norm1(x), self.norm2(y)), **kwargs)
        else:
            x = inp

            return self.fn(self.norm1(x), **kwargs)


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(

            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


### the TACO Cross Attention block that aims at modeling temporal-spatial dependencies simultaneously ###
class Cross_Attention(nn.Module):
    def __init__(self, dim, side_embedding_dim, heads, dim_head, side_dim_head, cross_heads, cross_dim_head,
                 dropout=0.):
        super().__init__()
        inner_dim = cross_dim_head * cross_heads

        project_out = not (cross_heads == 1 and cross_dim_head == dim)

        self.heads = cross_heads
        self.scale = (dim_head * side_dim_head) ** -0.5  # defined by head_dim of EEG and side_head_dim of PPS

        self.attend_chan = nn.Softmax(dim=-1)
        self.attend_token = nn.Softmax(dim=-2)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

        self.to_qkv_side = nn.Linear(side_embedding_dim, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(

            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, inp):
        x = inp[0]
        y = inp[1]

        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        qkv_side = self.to_qkv_side(y).chunk(3, dim=-1)
        qs, ks, vs = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv_side)

        sig_len = x.size()[1]
        # print("sig_len",sig_len)

        chan_dots = torch.matmul(q.transpose(-1, -2), ks) * self.scale
        token_dots = torch.matmul(qs, k.transpose(-1, -2)) * ((sig_len) ** -0.5)

        chan_attn = self.attend_chan(chan_dots)
        chan_attn = self.dropout(chan_attn)

        chan_out = torch.matmul(v, chan_attn)

        token_attn = self.attend_token(token_dots)
        token_attn = self.dropout(token_attn)

        out = torch.matmul(token_attn, chan_out)
        out = rearrange(out, 'b h n d -> b n (h d)')
        qs_res = rearrange(qs, 'b h n d -> b n (h d)')
        v_res = rearrange(v, 'b h n d -> b n (h d)')
        return v_res, out


class Transformer(nn.Module):
    def __init__(self, embedding_dim, side_embedding_dim, depth, side_depth, heads, side_heads, cross_heads, cross_dim_head,
                 dim_head, side_dim_head, mlp_dim, side_mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        self.side_layers = nn.ModuleList([])
        self.cross_modal = Cross_Attention(dim=embedding_dim, side_embedding_dim=side_embedding_dim, heads=heads,
                                           dim_head=dim_head, side_dim_head=side_dim_head, cross_heads=cross_heads,
                                           cross_dim_head=cross_dim_head, dropout=0.)
        self.cross_prenorm = PreNorm(embedding_dim, side_embedding_dim, self.cross_modal)
        self.cross_post = PreNorm(embedding_dim, side_embedding_dim, FeedForward(embedding_dim, mlp_dim, dropout=dropout))
        
        self.back_cross_modal = Cross_Attention(dim=side_embedding_dim, side_embedding_dim=embedding_dim, heads=heads,
                                           dim_head=dim_head, side_dim_head=side_dim_head, cross_heads=cross_heads,
                                           cross_dim_head=cross_dim_head, dropout=0.)
        self.back_cross_prenorm = PreNorm(side_embedding_dim, embedding_dim, self.back_cross_modal)
        self.back_cross_post = PreNorm(embedding_dim, embedding_dim, FeedForward(embedding_dim, mlp_dim, dropout=dropout))

        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(embedding_dim, side_embedding_dim, Attention(embedding_dim, heads=heads, dim_head=dim_head, dropout=dropout)),
                PreNorm(embedding_dim, side_embedding_dim, FeedForward(embedding_dim, mlp_dim, dropout=dropout))
            ]))

        for _ in range(side_depth):
            self.side_layers.append(nn.ModuleList([
                PreNorm(side_embedding_dim, side_embedding_dim,
                        Attention(side_embedding_dim, heads=side_heads, dim_head=side_dim_head, dropout=dropout)),
                PreNorm(side_embedding_dim, side_embedding_dim,
                        FeedForward(side_embedding_dim, side_mlp_dim, dropout=dropout))
            ]))

    def forward(self, inp):

        x = inp[0]  # EEG modal
        y = inp[1]  # PPS modal
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        for attn, ff in self.side_layers:
            y = attn(y) + y
            y = ff(y) + y
        
        # seperately pre-normalize x and y, then conduct TACO cross attention on x and y,
        # return "out" as fusion output and "vs",the multi-head value matrix as residual term
        vs, out = self.cross_prenorm((x, y))
        # residual connection
        k = vs + out
        # pre-normalize embedding k, and conduct Feedforward on it , finally conduct residual connection on k and output of Feedforward
        z = k + self.cross_post(k)
        
        
        
        
        
        return z


class ViT(nn.Module):
    def __init__(self, *, num_classes=2, height=9, width=9, embedding_dim=1024, side_dim=8, side_mid_dim=64,
                 side_embedding_dim=128, depth=8, side_depth=4, heads=8, side_heads=4, cross_heads=8, dim_head=64,
                 side_dim_head=32, cross_dim_head=64, mlp_dim=2048, side_mlp_dim=256, pool='cls', dropout=0.,
                 emb_dropout=0.):
        super().__init__()

        self.side_dim = side_dim
        self.height = height
        self.width = width
        patch_dim = height * width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(

            nn.Linear(patch_dim, embedding_dim)
        )
        self.to_patch_side_embedding = nn.Sequential(

            nn.Linear(side_dim, side_mid_dim),
            nn.Linear(side_mid_dim, side_embedding_dim)
        )
        self.position_enc = PositionalEncoding2d(height, width, n_position=200)
        self.side_position_enc = PositionalEncoding(side_embedding_dim, n_position=200)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim))
        self.side_cls_token = nn.Parameter(torch.randn(1, 1, side_embedding_dim))
        self.dropout = nn.Dropout(emb_dropout)
        self.transformer = Transformer(embedding_dim, side_embedding_dim, depth, side_depth, heads, side_heads,
                                       cross_heads, cross_dim_head, dim_head, side_dim_head, mlp_dim, side_mlp_dim,
                                       dropout=dropout)
        self.pool = pool
        self.to_latent = nn.Identity()
        self.laynorm = nn.LayerNorm(embedding_dim)
        self.side_laynorm = nn.LayerNorm(side_embedding_dim)
        self.mlp_head = nn.Sequential(
            nn.Linear((embedding_dim), num_classes)
        )

    def forward(self, img):
        img = img.permute(0, 2, 1)
        y = img[:, :, -self.side_dim:]
        img = img[:, :, :-self.side_dim]

        # reshape the 81 channel to 9X9 matrix
        x = img.reshape((img.shape[0], img.shape[1], self.height, self.width))
        # 2D position encoding
        x = self.position_enc(x)
        # reshape 9X9 matrix back into 81 channels
        x = x.reshape(x.shape[0], x.shape[1], self.height * self.width)

        # Linear embedding for EEG and PPS
        x = self.to_patch_embedding(x)
        y = self.to_patch_side_embedding(y)

        # concatenate class token as the first Token in sequence
        b, n, _ = x.shape
        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        x = self.dropout(x)
        x = self.laynorm(x)

        b_s, n_s, _s = y.shape
        side_cls_tokens = repeat(self.side_cls_token, '1 1 d -> b 1 d', b=b)
        y = torch.cat((side_cls_tokens, y), dim=1)
        y = self.side_position_enc(y)
        y = self.dropout(y)
        y = self.side_laynorm(y)

        # seperate transformer encoders and TACO Cross Attention block
        x = self.transformer((x, y))
        # integrating emotion-related information from multi tokens in Class Token
        x = x.mean(dim=1) if self.pool == 'mean' else x[:, 0,:]#b n c 

        x = self.to_latent(x)
        return self.mlp_head(x)


def train(epoch):
    Net = net
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(Net.parameters(), lr=learning_rate,weight_decay=weight_decay)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=-1)
    warmup_scheduler = warmup.UntunedLinearWarmup(optimizer)

    '''cuda accelerate'''
    device = ['gpu' if torch.cuda.is_available() else 'cpu']
    print(device)
    if device[0] == 'gpu':
        criterion.cuda()
        Net.cuda()

    print('start training...')
    for batch_idx, (data, target) in enumerate(train_loader):

        correct = 0
        if device[0] == 'gpu':
            data = data.cuda()
            target = target.cuda()

            data = data.to(torch.float32)
            target = target.to(torch.float32)

        optimizer.zero_grad()
        output = Net(data)
        loss = criterion(output, target.long())
        loss.backward()
        optimizer.step()
        with warmup_scheduler.dampening():
            lr_scheduler.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f},Accuracy: {}/{} ({:.0f}%)\n '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item(), correct, train_size,
                       100. * correct / train_size))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx * train_size) + ((epoch - 1) * len(train_loader.dataset)))
    print('training in this epoch completed ')


def test(real_test):
    network = net
    network.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss()
    '''cuda加速'''
    device = ['gpu' if torch.cuda.is_available() else 'cpu']
    print(device)
    if device == 'gpu':
        criterion.cuda()
        network.cuda()

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):

            data = data.to(torch.float32)
            target = target.to(torch.float32)

            if device[0] == 'gpu':
                data = data.cuda()
                target = target.cuda()
                data = data.to(torch.float32)
                target = target.to(torch.float32)

            output = network(data)

            test_loss += criterion(output, target.long())
            pred = output.data.max(1, keepdim=True)[1]

            correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    print("test loss:", test_loss)
    test_losses.append(test_loss)
    if real_test==True:
        print("running in test dataset with model trained on the last fold split:")
        print('\nin Test set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                                                                                         len(test_loader.dataset),
                                                                                         100. * correct / len(
                                                                                             test_loader.dataset)))
    if real_test==False:
        print('\nin Fold {},Test set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(fold,
                                                                                         test_loss, correct,
                                                                                         len(test_loader.dataset),
                                                                                         100. * correct / len(
                                                                                             test_loader.dataset)))

    #conf_matrix = confusion_matrix(pred, target.data.view_as(pred))
    #conf_matrix = conf_matrix.cpu()
    acc_this_fold = 100. * correct / len(test_loader.dataset)
    if real_test == False:
        # '''4.save model'''
        torch.save(network, 'deap_' + str(fold) + '.pt')
    return acc_this_fold


def confusion_matrix(preds, labels):
    for p, t in zip(preds.long(), labels.long()):
        conf_matrix[p, t] += 1
    return conf_matrix


if __name__ == "__main__":
    # get original data
    data_eeg = np.load("/root/autodl-tmp/datalist_real.npy")
    data_eog = np.load("/root/autodl-tmp/datalist_eog_deap.npy")
    data_emg = np.load("/root/autodl-tmp/datalist_emg_deap.npy")
    print(data_eeg.shape)# (32 40 60) , 81 128
    print(data_eog.shape)
    print(data_emg.shape)

    data_eeg = np.reshape(data_eeg, (1280,60, 81, 128))
    data_eog = np.reshape(data_eog, (1280,60, 4, 128))
    data_emg = np.reshape(data_emg, (1280,60, 4, 128))

    data = np.concatenate((data_eeg, data_eog), axis=2)
    data = np.concatenate((data, data_emg), axis=2)
    data = np.squeeze(data)
    print("total data", data.shape)
    y = np.load("/root/autodl-tmp/dataset/arousal.npy")
    y = np.reshape(y, (1280,60))

    y = y.astype(np.float64)
    data = data.astype(np.float64)

    # take test dataset out of whole dataset
   
    data, X_test, y, y_test = train_test_split(data, y, test_size=0.1, random_state=seed)# which is 1/32 
    print("X_test shape",X_test.shape)
    
    X = torch.Tensor(data)
    y = torch.Tensor(y)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    train_size =64
    test_size = 64
    accuracy_log = []

    for fold, (train_indices, valid_indices) in enumerate(kf.split(X)):
        train_data, validation_data = X[train_indices], X[valid_indices]
        train_labels, validation_labels = y[train_indices], y[valid_indices]
        print("for_train", len(train_data))
        print("for_valid", len(validation_data))
        
        
        # reshape datas into shape: samples, channel ,timestamp
        train_data = np.reshape(train_data, (61440, 89, 128))#30*2400
        train_labels = np.reshape(train_labels, (61440,))


        
        validation_data=np.reshape(validation_data,(7680,89,128))
        validation_labels=np.reshape(validation_labels,(7680,))


        train_dataset = TensorDataset(train_data, train_labels)
        test_dataset = TensorDataset(validation_data, validation_labels)
        train_loader = DataLoader(train_dataset, batch_size=train_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=False)

        log_interval = 50
        n_epochs=100
        train_losses = []
        train_counter = []
        test_losses = []
        test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

        net=ViT(
            num_classes = 2,
            height=9,
            width=9,
            embedding_dim=128,
            side_dim=8,
            side_mid_dim=64,#for linear embedding 
            side_embedding_dim=128,#for linear embedding 


            depth = depth,
            side_depth=side_depth,
            heads = 4,
            side_heads=4,
            cross_heads=4,

            dim_head=32,
            side_dim_head=32,
            cross_dim_head=32,


            mlp_dim = 128,# used in Feedforward 
            side_mlp_dim=128,# used in Feedforward  
            dropout = 0.2,
            emb_dropout = 0.2,
            
        )
        for name in net.state_dict():
          print(name)


        for epoch in range(1, n_epochs + 1):
          train(epoch)
          acc_this_fold=test(real_test=False)
        accuracy_log.append(acc_this_fold)

        #rint("conf_mat",conf_matrix)
        test_losses=torch.tensor(test_losses,device="cpu")
        import matplotlib.pyplot as plt
        fig = plt.figure()
        plt.plot(train_counter, train_losses, color='blue')
        print(np.array(test_counter[1:]).shape)
        print(np.array(test_losses).shape)
        plt.scatter(np.array(test_counter[1:]), np.array(test_losses), color='red')
        plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
        plt.xlabel('number of training examples seen')
        plt.ylabel('negative log likelihood loss')
        plt.savefig("train loss of fold"+str(fold))
        plt.show()
   
    print("acc in 31 fold cross validation:", accuracy_log)
    print("average acc in 31-fold cross subject validation",torch.mean(accuracy_log))
    
    





1.11.0+cu113
11.3
weight_decay 0
learning_rate 0.001
depth 3
side_depth 3
random seed: 778539
(76800, 81, 128, 1)
(76800, 4, 128, 1)
(76800, 4, 128, 1)
total data (1280, 60, 89, 128)
X_test shape (128, 60, 89, 128)
for_train 1024
for_valid 128
cls_token
side_cls_token
to_patch_embedding.0.weight
to_patch_embedding.0.bias
to_patch_side_embedding.0.weight
to_patch_side_embedding.0.bias
to_patch_side_embedding.1.weight
to_patch_side_embedding.1.bias
position_enc.pos_table
side_position_enc.pos_table
transformer.layers.0.0.norm1.weight
transformer.layers.0.0.norm1.bias
transformer.layers.0.0.norm2.weight
transformer.layers.0.0.norm2.bias
transformer.layers.0.0.fn.to_qkv.weight
transformer.layers.0.1.norm1.weight
transformer.layers.0.1.norm1.bias
transformer.layers.0.1.norm2.weight
transformer.layers.0.1.norm2.bias
transformer.layers.0.1.fn.net.0.weight
transformer.layers.0.1.fn.net.0.bias
transformer.layers.0.1.fn.net.3.weight
transformer.layers.0.1.fn.net.3.bias
transformer.layers.1.0.nor